# Quantum Analysis

In [2]:
import os
import numpy as np

# Load EEG data
EEG_data = np.load('/home/vincent/AAA_projects/MVCS/Neuroscience/eeg_data_with_channels.npy', allow_pickle=True)

# Adjust the list to match your data's channels
eeg_channels = ['Fp1', 'Fpz', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6',
                'M1', 'T7', 'C3', 'Cz', 'C4', 'T8', 'M2', 'CP5', 'CP1', 'CP2', 'CP6',
                'P7', 'P3', 'Pz', 'P4', 'P8', 'POz', 'O1', 'Oz', 'O2']

# Simulate some EEG data for the sake of this example
# Normally, you'd import or read this data from a file.
num_samples = 1000
num_channels = len(eeg_channels)
EEG_data = np.random.rand(num_samples, num_channels)

def construct_density_matrix(data):
    """
    Constructs a density matrix from data.
    This is a metaphorical interpretation of the EEG data.
    """
    prob_dist = data / np.sum(data)
    return np.outer(prob_dist, prob_dist)

def calculate_coherence(rho):
    """
    Calculates coherence from a density matrix.
    """
    # Eigenvalues of the density matrix give the probabilities
    eigenvalues = np.linalg.eigvals(rho)
    # Quantum coherence is given by the l1 norm of the matrix after diagonalizing it.
    l1_norm = np.sum(np.abs(eigenvalues)) - np.max(np.abs(eigenvalues))
    return l1_norm

# Calculate "quantum coherence" for each EEG sample
coherences = []

for sample in range(num_samples):
    rho = construct_density_matrix(EEG_data[sample])
    coherence = calculate_coherence(rho)
    coherences.append(coherence)

# Save the results
output_path = "/home/vincent/AAA_projects/MVCS/Neuroscience/Analysis/Quantum_Analysis/coherences.npy"
os.makedirs(os.path.dirname(output_path), exist_ok=True)
np.save(output_path, coherences)


In [3]:
# Print the calculated coherences
print("Quantum Coherence Values:")
for idx, coherence in enumerate(coherences):
    print(f"Sample {idx+1}: {coherence}")

Quantum Coherence Values:
Sample 1: 6.938893903907228e-18
Sample 2: 6.938893903907228e-18
Sample 3: 6.938893903907228e-18
Sample 4: 1.3877787807814457e-17
Sample 5: 1.3877787807814457e-17
Sample 6: 2.0816681711721685e-17
Sample 7: 1.3877787807814457e-17
Sample 8: 1.3877787807814457e-17
Sample 9: 1.3877787807814457e-17
Sample 10: 6.938893903907228e-18
Sample 11: 1.3877787807814457e-17
Sample 12: 1.3877787807814457e-17
Sample 13: 1.3877787807814457e-17
Sample 14: 6.938893903907228e-18
Sample 15: 6.938893903907228e-18
Sample 16: 1.3877787807814457e-17
Sample 17: 2.0816681711721685e-17
Sample 18: 1.3877787807814457e-17
Sample 19: 1.3877787807814457e-17
Sample 20: 1.3877787807814457e-17
Sample 21: 2.0816681711721685e-17
Sample 22: 1.3877787807814457e-17
Sample 23: 1.3877787807814457e-17
Sample 24: 1.3877787807814457e-17
Sample 25: 1.3877787807814457e-17
Sample 26: 1.3877787807814457e-17
Sample 27: 1.3877787807814457e-17
Sample 28: 6.938893903907228e-18
Sample 29: 2.0816681711721685e-17
Samp

# Quantum-like Entanglement

In [4]:
import numpy as np
import os

# Example EEG data
num_samples = 1000
num_channels = len(eeg_channels)
EEG_data = np.random.rand(num_samples, num_channels)

def construct_joint_density_matrix(data1, data2):
    """
    Constructs a joint density matrix from two datasets.
    """
    prob_dist1 = data1 / np.sum(data1)
    prob_dist2 = data2 / np.sum(data2)
    joint_prob_dist = np.outer(prob_dist1, prob_dist2)
    return joint_prob_dist

def von_neumann_entropy(rho):
    """
    Calculates the Von Neumann entropy from a density matrix.
    """
    eigenvalues = np.linalg.eigvals(rho)
    eigenvalues = eigenvalues[eigenvalues > 0]  # Remove non-positive eigenvalues
    entropy = -np.sum(eigenvalues * np.log2(eigenvalues))
    return entropy

coherences = []

# Calculate the quantum-like coherence measure for each EEG sample
for sample in range(num_samples):
    entropy_values = []
    
    for channel in range(num_channels):
        rho = construct_density_matrix(EEG_data[sample, channel])
        entropy = von_neumann_entropy(rho)
        entropy_values.append(entropy)
        
    joint_entropies = []
    
    for ch1 in range(num_channels):
        for ch2 in range(ch1+1, num_channels):  # Pairwise combinations
            joint_rho = construct_joint_density_matrix(EEG_data[sample, ch1], EEG_data[sample, ch2])
            joint_entropy = von_neumann_entropy(joint_rho)
            joint_entropies.append(joint_entropy)
    
    # Define coherence as average entropy difference
    average_entropy = np.mean(entropy_values)
    average_joint_entropy = np.mean(joint_entropies)
    coherence = average_joint_entropy - 2 * average_entropy  # Factor of 2 due to bipartite system
    coherences.append(coherence)

coherences = []

# Loop over each EEG channel
for channel in range(num_channels):
    entropy_values = []

    # Calculate the entropy for the current channel
    for sample in range(num_samples):
        rho = construct_density_matrix(EEG_data[sample, channel])
        entropy = von_neumann_entropy(rho)
        entropy_values.append(entropy)

    # Calculate the average entropy for the current channel
    average_entropy = np.mean(entropy_values)

    # Calculate coherence for the current channel
    coherence = average_joint_entropy - 2 * average_entropy  # Similar to previous formula
    coherences.append(coherence)

# Save the results
output_path = "/home/vincent/AAA_projects/MVCS/Neuroscience/Analysis/Quantum_Analysis/coherences.npy"
os.makedirs(os.path.dirname(output_path), exist_ok=True)
np.save(output_path, coherences)

# Print the calculated coherences
print("Quantum Coherence Values:")
for idx, coherence in enumerate(coherences):
    print(f"Sample {idx+1}: {coherence}")


Quantum Coherence Values:
Sample 1: 0.0
Sample 2: 0.0
Sample 3: 0.0
Sample 4: 0.0
Sample 5: 0.0
Sample 6: 0.0
Sample 7: 0.0
Sample 8: 0.0
Sample 9: 0.0
Sample 10: 0.0
Sample 11: 0.0
Sample 12: 0.0
Sample 13: 0.0
Sample 14: 0.0
Sample 15: 0.0
Sample 16: 0.0
Sample 17: 0.0
Sample 18: 0.0
Sample 19: 0.0
Sample 20: 0.0
Sample 21: 0.0
Sample 22: 0.0
Sample 23: 0.0
Sample 24: 0.0
Sample 25: 0.0
Sample 26: 0.0
Sample 27: 0.0
Sample 28: 0.0
Sample 29: 0.0
Sample 30: 0.0
Sample 31: 0.0
Sample 32: 0.0
